In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

## Creación de datos de ejemplo

In [ ]:
np.random.seed(42)

# Dataset 1: Signos vitales (frecuencia alta - cada minuto)
inicio = pd.Timestamp('2025-01-01 08:00:00')
timestamps_vitales = pd.date_range(start=inicio, periods=120, freq='1min')

df_vitales = pd.DataFrame({
    'timestamp': timestamps_vitales,
    'paciente_id': 'P001',
    'HR': np.random.randint(60, 100, 120),
    'SBP': np.random.randint(110, 140, 120),
    'SpO2': np.random.randint(95, 100, 120)
})

print("=== Signos Vitales (cada minuto) ===")
print(df_vitales.head(10))
print(f"Total registros: {len(df_vitales)}")

In [ ]:
# Dataset 2: Administración de medicamentos (frecuencia baja - cada hora)
timestamps_medicamentos = pd.date_range(start=inicio, periods=3, freq='1h')

df_medicamentos = pd.DataFrame({
    'timestamp': timestamps_medicamentos,
    'paciente_id': 'P001',
    'medicamento': ['Dopamina', 'Norepinefrina', 'Dopamina'],
    'dosis_mg': [5.0, 2.5, 5.0]
})

print("\n=== Administración de Medicamentos (cada hora) ===")
print(df_medicamentos)

In [ ]:
# Dataset 3: Eventos clínicos
df_eventos = pd.DataFrame({
    'timestamp': [inicio + timedelta(minutes=30), 
                  inicio + timedelta(minutes=75)],
    'paciente_id': ['P001', 'P001'],
    'evento': ['Hipotensión detectada', 'Estabilización lograda']
})

print("\n=== Eventos Clínicos ===")
print(df_eventos)

## 1. Concatenación vertical (concat) - Apilar filas

In [ ]:
# Crear datos de múltiples pacientes
df_vitales_p002 = pd.DataFrame({
    'timestamp': pd.date_range(start=inicio, periods=60, freq='1min'),
    'paciente_id': 'P002',
    'HR': np.random.randint(70, 110, 60),
    'SBP': np.random.randint(100, 130, 60),
    'SpO2': np.random.randint(93, 99, 60)
})

# Concatenar verticalmente (añadir más filas)
df_vitales_todos = pd.concat([df_vitales, df_vitales_p002], ignore_index=True)

print("=== Concatenación vertical: múltiples pacientes ===")
print(f"Paciente P001: {len(df_vitales)} registros")
print(f"Paciente P002: {len(df_vitales_p002)} registros")
print(f"Total concatenado: {len(df_vitales_todos)} registros")
print("\nPrimeros registros:")
print(df_vitales_todos.head())
print("\nÚltimos registros:")
print(df_vitales_todos.tail())

## 2. Concatenación horizontal (concat axis=1) - Añadir columnas

In [ ]:
# Crear datos adicionales con mismo índice
df_labs = pd.DataFrame({
    'timestamp': timestamps_vitales[:10],
    'lactato': np.random.uniform(0.5, 2.5, 10),
    'glucosa': np.random.uniform(80, 120, 10)
})

df_vitales_subset = df_vitales.iloc[:10].copy()

# Concatenar horizontalmente (añadir columnas)
df_completo = pd.concat([df_vitales_subset, df_labs[['lactato', 'glucosa']]], axis=1)

print("=== Concatenación horizontal: añadir variables ===")
print(df_completo.head())

## 3. Merge con timestamps exactos (inner join)

In [ ]:
# Merge basado en timestamp exacto
df_merged_inner = pd.merge(
    df_vitales,
    df_medicamentos,
    on=['timestamp', 'paciente_id'],
    how='inner'  # Solo registros que coinciden en ambos
)

print("=== INNER MERGE: Solo timestamps que coinciden exactamente ===")
print(f"Registros resultantes: {len(df_merged_inner)}")
print(df_merged_inner)

## 4. Merge con outer join - Mantener todos los registros

In [ ]:
# Outer merge: mantener todos los registros
df_merged_outer = pd.merge(
    df_vitales,
    df_medicamentos,
    on=['timestamp', 'paciente_id'],
    how='outer'  # Todos los registros de ambos DataFrames
)

print("=== OUTER MERGE: Todos los registros ===")
print(f"Registros resultantes: {len(df_merged_outer)}")
print("\nPrimeros registros:")
print(df_merged_outer.head(10))
print("\nRegistros con medicamentos:")
print(df_merged_outer[df_merged_outer['medicamento'].notna()])

## 5. Merge asof - Para diferentes frecuencias de muestreo

In [ ]:
# merge_asof: une por timestamp más cercano (ideal para diferentes frecuencias)
# Importante: DataFrames deben estar ordenados por timestamp

df_vitales_sorted = df_vitales.sort_values('timestamp')
df_medicamentos_sorted = df_medicamentos.sort_values('timestamp')

df_merged_asof = pd.merge_asof(
    df_vitales_sorted,
    df_medicamentos_sorted,
    on='timestamp',
    by='paciente_id',
    direction='backward'  # Usar el medicamento más reciente administrado
)

print("=== MERGE ASOF: Unir por timestamp más cercano ===")
print(f"Registros resultantes: {len(df_merged_asof)}")
print("\nPrimeros 20 registros (muestra administración de medicamentos):")
print(df_merged_asof[['timestamp', 'HR', 'SBP', 'medicamento', 'dosis_mg']].head(20))
print("\nAlrededor del segundo medicamento (minuto 60):")
print(df_merged_asof[['timestamp', 'HR', 'SBP', 'medicamento', 'dosis_mg']].iloc[58:63])

## 6. Left merge - Mantener todos los signos vitales

In [ ]:
# Left merge: mantener todos los registros de signos vitales
df_merged_left = pd.merge(
    df_vitales,
    df_eventos,
    on=['timestamp', 'paciente_id'],
    how='left'  # Mantener todos de la izquierda (vitales)
)

print("=== LEFT MERGE: Todos los signos vitales con eventos ===")
print(f"Registros resultantes: {len(df_merged_left)}")
print("\nRegistros donde hay eventos:")
print(df_merged_left[df_merged_left['evento'].notna()])

## 7. Join - Unir por índice

In [ ]:
# Establecer timestamp como índice
df_vitales_idx = df_vitales.set_index('timestamp')
df_medicamentos_idx = df_medicamentos.set_index('timestamp')

# Join usando el índice
df_joined = df_vitales_idx.join(
    df_medicamentos_idx[['medicamento', 'dosis_mg']], 
    how='left',
    rsuffix='_med'
)

print("=== JOIN por índice (timestamp) ===")
print(f"Registros resultantes: {len(df_joined)}")
print("\nRegistros con medicamentos:")
print(df_joined[df_joined['medicamento'].notna()])

## 8. Error común: Índices duplicados

In [ ]:
# Crear DataFrame con índices duplicados (error común)
df_con_duplicados = pd.DataFrame({
    'timestamp': [inicio, inicio, inicio + timedelta(minutes=1)],
    'paciente_id': ['P001', 'P001', 'P001'],
    'HR': [75, 78, 80]
})

print("=== DataFrame con timestamps duplicados ===")
print(df_con_duplicados)

# Verificar duplicados
duplicados = df_con_duplicados.duplicated(subset=['timestamp', 'paciente_id'], keep=False)
print(f"\nRegistros duplicados: {duplicados.sum()}")
print(df_con_duplicados[duplicados])

# Solución: eliminar duplicados o promediar
df_sin_duplicados = df_con_duplicados.groupby(['timestamp', 'paciente_id']).mean().reset_index()
print("\nSolución - Promediar duplicados:")
print(df_sin_duplicados)

## 9. Error común: Columnas inconsistentes

In [ ]:
# DataFrames con columnas diferentes
df1 = pd.DataFrame({
    'paciente_id': ['P001', 'P002'],
    'HR': [75, 80],
    'SBP': [120, 130]
})

df2 = pd.DataFrame({
    'paciente_id': ['P003', 'P004'],
    'HR': [85, 90],
    'DBP': [80, 85]  # Columna diferente
})

print("=== Concatenar con columnas inconsistentes ===")
print("DataFrame 1:")
print(df1)
print("\nDataFrame 2:")
print(df2)

# Concatenar: crea NaN donde faltan columnas
df_concat_inconsistente = pd.concat([df1, df2], ignore_index=True)
print("\nResultado (con NaN):")
print(df_concat_inconsistente)

# Solución: especificar columnas comunes
columnas_comunes = ['paciente_id', 'HR']
df_concat_limpio = pd.concat([df1[columnas_comunes], df2[columnas_comunes]], ignore_index=True)
print("\nSolución - Solo columnas comunes:")
print(df_concat_limpio)

## 10. Caso práctico completo: Unir múltiples fuentes

In [ ]:
# Combinar signos vitales, medicamentos y eventos
# Paso 1: Merge asof para medicamentos (diferentes frecuencias)
df_paso1 = pd.merge_asof(
    df_vitales.sort_values('timestamp'),
    df_medicamentos.sort_values('timestamp'),
    on='timestamp',
    by='paciente_id',
    direction='backward'
)

# Paso 2: Left merge para eventos
df_final = pd.merge(
    df_paso1,
    df_eventos,
    on=['timestamp', 'paciente_id'],
    how='left'
)

print("=== Dataset integrado final ===")
print(f"Total registros: {len(df_final)}")
print("\nEstructura:")
print(df_final.info())
print("\nMuestra de datos:")
print(df_final[['timestamp', 'HR', 'SBP', 'SpO2', 'medicamento', 'dosis_mg', 'evento']].head(15))
print("\nRegistros con eventos clínicos:")
print(df_final[df_final['evento'].notna()][['timestamp', 'HR', 'SBP', 'medicamento', 'evento']])

## Resumen

### Tipos de unión:

1. **concat()**: 
   - Vertical (axis=0): apilar filas (múltiples pacientes)
   - Horizontal (axis=1): añadir columnas

2. **merge()**:
   - `inner`: solo registros que coinciden
   - `outer`: todos los registros
   - `left`: todos de la izquierda
   - `right`: todos de la derecha

3. **merge_asof()**: 
   - Ideal para diferentes frecuencias de muestreo
   - Une por timestamp más cercano

4. **join()**: 
   - Une por índice
   - Más conciso que merge para uniones por índice

### Errores comunes:
- **Índices duplicados**: verificar con `duplicated()`, resolver con `groupby()` o `drop_duplicates()`
- **Columnas inconsistentes**: especificar columnas comunes o manejar NaN
- **Timestamps desalineados**: usar `merge_asof` para diferentes frecuencias

### Buenas prácticas:
- Siempre verificar duplicados antes de merge
- Ordenar por timestamp antes de merge_asof
- Validar tamaño de resultado después de merge
- Documentar tipo de merge usado y razón